In [1]:
import os

# 이 파일과 동일한 폴더에 자신의 gemini API가 텍스트 형태로 담긴 gemini_api.key 파일을 둘 것.
with open('openai_api.key', 'r') as f:
    api_key = f.read()

os.environ["OPENAI_API_KEY"] = api_key

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

# 모델 초기화
model = ChatOpenAI(model="gpt-4o-mini")

In [3]:
from langchain_core.documents import Document
import json

# 원본을 쓸 경우 0, 요약본을 쓸 경우 1
# !!! 이후 코드에 전부 적용되도록 작성했으므로 여기만 바꾸면 됨 !!!
select_summary = 0

suffix = suffix = '_summary' if select_summary == 1 else ''
with open('naver_stock_news/today_news' + suffix + '.json', 'r') as f:
    articles = json.load(f)

docs = list()

for a in articles:
    if select_summary == 1: detail = a.pop('summary', '')
    else: detail = a.pop('content', '')
    docs.append(Document(page_content=detail + '\n\n' + 'metadata=' + str(a), metadata=a))

docs[:5]

[Document(metadata={'title': '"가장 높은 220달러 나왔다"…50% 이상 급등 \'베팅\'', 'url': 'https://n.news.naver.com/mnews/article/215/0001189038', 'date': '2024-11-25 10:54:56'}, page_content='\n엔비디아의 3분기 실적발표 이벤트 이후 글로벌 투자은행(IB)들이 목표가를 일제히 상향 조정하며 강세론을 고수하고 있다. 23일(현지시간) 비즈니스 인사이더에 따르면 JP모간은 엔비디아의 목표가를 기존 155달러에서 170달러로 올리며 "엔비디아의 하드웨어와 소프트웨어 플랫폼은 경쟁사보다 1~2단계 앞서 있고 앞으로 제품 출시와 세분화 전략으로 격차가 더 크게 벌어질 것"이라고 강조했다. 골드만삭스는 엔비디아의 목표가를 기존 150달러에서 160달러로 상향 조정하며 엔비디아가 내년에 2,000억 달러 이상의 매출을 기록할 것으로 전망했다. 또한 대규모 자사주 매입 프로그램이 주가 상승의 원동력이 될 것으로 예상하며 엔비디아가 오는 2026년까지 총 1,810억 규모의 자사주 매입에 나설 것으로 내다봤다. 로젠블랫은 글로벌 IB 가운데 가장 높은 목표가인 220달러를 제시했다. 이는 엔비디아가 향후 50% 이상 급등할 수 있음을 시사한다. 또한 밸류에이션 우려로 투자의견을 \'중립(Neutral)\'으로 제시한 DA 데이비드슨도 목표가를 기존 90에서 135달러로 상향 조정했다. 비즈니스 인사이더에 따르면 글로벌 IB들의 엔비디아 평균 목표가는 3분기 실적발표 이후 168달러로 상향 조정됐다. 이는 실적발표 전에 집계된 150달러보다 12%가량 높은 수준이다. (사진=비즈니스 인사이더)\n\n\nmetadata={\'title\': \'"가장 높은 220달러 나왔다"…50% 이상 급등 \\\'베팅\\\'\', \'url\': \'https://n.news.naver.com/mnews/article/215/0001189038\', \'date\': 

In [4]:
# 요약된 것을 쓰는 경우 청킹은 굳이 수행하지 않는다.
if select_summary == 1: splits = docs

# 원본을 썼다면 청킹을 수행한다.
else:
    from langchain.text_splitter import RecursiveCharacterTextSplitter

    recursive_text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=100,
        chunk_overlap=2,
        length_function=len,
        is_separator_regex=False,
    )

    splits = recursive_text_splitter.split_documents(docs)
    splits[:5]

In [5]:
# 만약 그대로 쓰고 싶다면 추가할 것.
splits = docs

In [6]:
from langchain_openai import OpenAIEmbeddings

# OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

In [7]:
import faiss
from langchain_community.vectorstores import FAISS

vector_store = FAISS.from_documents(documents=splits, embedding=embeddings)

In [8]:
# 원본 기준 참고할 Document 개수는 10개, 요약본 기준 3개 정도로 한다.
quantity = 3 if select_summary == 1 else 10
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": quantity})

In [9]:
# 시스템 프롬프트 정의
system_prompt = """
반드시 이미 주어져 있는 맥락만을 기반으로 해서 사용자의 질문에 대해 한국어로 답변하도록 한다.
이때 다음과 같이 메타데이터를 출력한다. 만약 사용자의 질문에 대해 답변할 때 참조할 데이터가 없다면 메타데이터를 출력하지 말자.

- 메타데이터는 맥락의 가장 아래줄에 딕셔너리 형태로 존재한다.
- 메타데이터는 title, url, date 속성이 있다.
- 답변을 다 한 후, 마지막에 이 메타데이터를 다음과 같은 형식으로 출력하자.
    
    기사 제목: title
    기사 주소: url
    작성 날짜: date
"""

In [10]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# 프롬프트 템플릿 정의
contextual_prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", "Context: {context}\\n\\nQuestion: {question}")
])

In [11]:
class DebugPassThrough(RunnablePassthrough):
    def invoke(self, *args, **kwargs):
        output = super().invoke(*args, **kwargs)
        print("Debug Output:", output)
        return output
# 문서 리스트를 텍스트로 변환하는 단계 추가
class ContextToText(RunnablePassthrough):
    def invoke(self, inputs, config=None, **kwargs):  # config 인수 추가
        # context의 각 문서를 문자열로 결합
        context_text = "\n".join([doc.page_content for doc in inputs["context"]])
        return {"context": context_text, "question": inputs["question"]}

# RAG 체인에서 각 단계마다 DebugPassThrough 추가
# 아래에 파이프(|)로 연결된 클래스들을 순차적으로 통과한다.
rag_chain_debug = {
    "context": retriever,                    # 컨텍스트를 가져오는 retriever
    "question": DebugPassThrough()        # 사용자 질문이 그대로 전달되는지 확인하는 passthrough
} | DebugPassThrough() | ContextToText() | contextual_prompt | model


In [ ]:
# 이제 OpenAI를 API로 연결해서 대화를 진행해보자.
while 1: 
	print("========================")
	query = input("질문을 입력하세요: ")
	response = rag_chain_debug.invoke(query)
	print("Final Response:")
	print(response.content)

Debug Output: 엔비디아 관련 소식을 알려줘
Debug Output: {'context': [Document(metadata={'title': '"HBM 승인 위해 작업 중"…젠슨 황 발언에 삼성전자 \'강세\'', 'url': 'https://n.news.naver.com/mnews/article/015/0005060897', 'date': '2024-11-25 09:15:12'}, page_content='\n사진=AP삼성전자가 장 초반 강세를 보이고 있다. 엔비디아가 삼성전자의 고대역폭메모리(HBM) 승인을 위해 최대한 빨리 작업하고 있다는 소식이 전해지면서다. 25일 오전 9시10분 현재 삼성전자는 전 거래일 대비 1600원(2.86%) 오른 5만7600원에 거래되고 있다. 주가는 장중 5만7700원까지 치솟기도 했다. HBM 관련 기대감에 투자심리가 개선된 것으로 풀이된다. 블룸버그통신에 따르면 23일(현지시간) 젠슨 황 엔비디아 최고경영자(CEO)는 홍콩 과학기술대 명예박사 학위 수여식에서 "삼성전자로부터 5세대 HBM인 HBM3E 8단과 12단 모두 납품받는 방안을 검토하고 있다고 설명했다. 또 납품 승인을 위해 최대한 빨리 작업 중이라고 밝혔다. 현재 SK하이닉스가 엔비디아에 사실상 HBM을 독점 공급하고 있다. 삼성전자는 퀄 테스트(품질 검증)를 통과하지 못한 것으로 알려졌다. 이 때문에 삼성전자는 인공지능(AI) 열풍에서 소외됐고, 주가가 내리막길을 걸었다. 블룸버그는 황 CEO가 최근 3분기(8∼10월) 실적 발표 후 콘퍼런스콜에서 메모리 공급업체로 SK하이닉스·마이크론 등을 언급하면서도 삼성전자는 거론하지 않았다고 전했다. 다만 엔비디아 입장에서도 원가 절감, 가격 협상력 등을 고려하면 삼성전자의 HBM이 필요한 상황이다. 삼성전자는 지난달 31일 3분기 실적 콘퍼런스콜에서 "현재 HBM3E 8단·12단 모두 양산 판매 중으로, 주요 고객사 품질 테스트 과정상 중요한 단계를 완료하는 유의미한 진전을 확보했고 4분기 안에 판매 확대